Importación de bibliotecas

In [1]:
from boxoffice_api import BoxOffice
import rottentomatoes as rt
import datetime
import pandas as pd
import numpy as np 
import psycopg2
import psycopg2.extras as extras
import os
from dotenv import load_dotenv

Funciones

In [2]:
def execute_values(conn, df, table): 
  
    tuples = [tuple(x) for x in df.to_numpy()] 
  
    cols = ','.join(list(df.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    print("the dataframe is inserted") 
    cursor.close()

def getRottenRating(peli):
    try:
        ptg = rt.tomatometer(peli)
    except:
        ptg = 0
    return ptg

def getAudienceRating(peli):
    try:
        ptg = rt.audience_score(peli)
    except:
        ptg = 0
    return ptg

def getGenre(peli):
    try:
        genre = rt.genres(peli)
    except:
        genre = ['No data']
        
    genre_f = '; '.join(genre)
    return genre_f

Obtención de datos de la API

In [3]:
#Obtengo el password para la API
with open("rotten_api.txt",'r') as f:
    pwd= f.read()

In [4]:
#Inicialización de la API
box_office = BoxOffice(api_key=pwd,outputformat="DF")

In [5]:
#Seteo de datos de fecha
hoy = datetime.date.today()

In [6]:
#Obtengo los datos de la API
df = box_office.get_monthly(year=hoy.year,month=hoy.month)
cols = df.columns[:-2]
df = df[cols]
df = df.rename(columns={'Total Gross':'Total_Gross','Release Date':'Release_Date'})
df = df.head(10)

KeyboardInterrupt: 

Agrego datos extra al DF

In [ ]:
df['Year'] = hoy.year
df['Month'] = hoy.month

df['RottenTomatoes_Score'] = [getRottenRating(x) for x in df['Release']]
df['Audience_Score'] = [getAudienceRating(x) for x in df['Release']]
df['Genre'] = [getGenre(x) for x in df['Release']]

Conexión a DB

In [ ]:
#Obtengo el password para Redshift
with open("redshift_info.txt",'r') as f:
    pwd= f.read()

In [ ]:
#Conectando a la DB
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname='data-engineer-database',
        user='pdiazmedin_coderhouse',
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


Creación de la tabla en Redshift

In [ ]:
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS peliculas
        ( 
		Rank INTEGER,
		Release VARCHAR(255),
		Gross VARCHAR(255),
		Theaters VARCHAR(255),
		Total_Gross VARCHAR(255),
		Release_Date VARCHAR(255),
		Distributor VARCHAR(255),
		Year INTEGER,
		Month INTEGER,
		RottenTomatoes_Score INTEGER,
		Audience_Score INTEGER,
		Genre VARCHAR(255)
        )
    """)
    conn.commit()


Insertando datos en la tabla

In [ ]:
execute_values(conn, df, 'peliculas') 

Error: column "genre" is of type character varying but expression is of type text[]
HINT:  You will need to rewrite or cast the expression.



1

In [ ]:
display(df)

,Rank,Release,Gross,Theaters,Total_Gross,Release_Date,Distributor,Year,Month,RottenTomatoes_Score,Audience_Score,Genre
0,1,Dune: Part Two,"$207,528,001","4,074","$207,528,001",Mar 1,Warner Bros.,2024,3,92,95,"[Sci-Fi, Adventure, Action, Fantasy, Drama]"
1,2,Kung Fu Panda 4,"$109,913,285","4,067","$109,913,285",Mar 8,Universal Pictures,2024,3,72,86,"[Kids & Family, Comedy, Adventure, Animation, ..."
2,3,Imaginary,"$19,488,508","3,118","$19,488,508",Mar 8,Lionsgate Films,2024,3,25,53,"[Horror, Mystery & Thriller]"
3,4,Bob Marley: One Love,"$18,222,746","3,597","$93,563,826",Feb 14,Paramount Pictures,2024,3,43,92,"[Biography, Drama, Music]"
4,5,Cabrini,"$13,415,888","2,850","$13,415,888",Mar 8,Angel Studios,2024,3,90,98,"[Biography, History, Drama]"
5,6,Ordinary Angels,"$9,405,238","3,020","$18,116,360",Feb 23,-,2024,3,86,99,[Drama]
6,7,Arthur the King,"$8,215,103","3,003","$8,215,103",Mar 15,Lionsgate Films,2024,3,67,97,"[Drama, Adventure]"
7,8,Madame Web,"$6,328,497","4,013","$43,570,013",Feb 14,Columbia Pictures,2024,3,12,57,"[Action, Adventure, Mystery & Thriller]"
8,9,The Chosen: S4 Episodes 7-8,"$6,247,415","2,215","$7,011,749",Feb 29,Fathom Events,2024,3,65,56,[Comedy]
9,10,Migration,"$5,614,750","3,839","$126,571,630",Dec 22,Universal Pictures,2024,3,73,88,"[Kids & Family, Comedy, Adventure, Animation]"
